# AMGeO 

In [ ]:
from AMGeO.api import AMGeOApi

In [ ]:
api = AMGeOApi()
api

AMGeO API instance
current configuration: default
current output directory: /workspaces/AMGeO-dev/amgeo_v2.0.1_output

In [ ]:
api.get_output_dir()

AMGeO API instance
current configuration: default
current output directory: /workspaces/AMGeO-dev/amgeo_v2.0.1_output